In [1]:
import numpy as np
import pandas as pd

# Load Data (replace this with actual CSV file or DataFrame)
data = pd.read_csv('merged_properties.csv')  # Replace 'your_file.csv' with your actual file path
# Features and targets
# X = data.drop(columns=['c44', 'e15', 'q15', 'μ11', 'ϵ11', 'α11', 'ρ']).values
X = data[['vf', 'c55e', 'e15e', 'q15e', 'ϵ11e', 'μ11e', 'α11e', 'ρe', 'c55f', 'e15f', 'q15f', 'ϵ11f', 'μ11f', 'α11f', 'ρf']].values
y = data[['c44', 'e15', 'q15', 'μ11', 'ϵ11', 'α11', 'ρ']].values


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, PowerTransformer, PolynomialFeatures

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling for training and testing sets
feature_robust_scaler = RobustScaler()
X_train_robust_scaled = feature_robust_scaler.fit_transform(X_train)
X_test_robust_scaled = feature_robust_scaler.transform(X_test)

feature_power_scaler = PowerTransformer(method='yeo-johnson')
X_train_power_scaled = feature_power_scaler.fit_transform(X_train)
X_test_power_scaled = feature_power_scaler.transform(X_test)

poly = PolynomialFeatures(degree=2, include_bias=False)
feature_poly_robust_scaler = RobustScaler()
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
X_train_poly_robust_scaled = feature_poly_robust_scaler.fit_transform(X_train_poly)
X_test_poly_robust_scaled = feature_poly_robust_scaler.transform(X_test_poly)

# Target scaling for training and testing sets
target_robust_scaler = RobustScaler()
y_train_robust_scaled = target_robust_scaler.fit_transform(y_train)
y_test_robust_scaled = target_robust_scaler.transform(y_test)

target_power_scaler = PowerTransformer(method='yeo-johnson')
y_train_power_scaled = target_power_scaler.fit_transform(y_train)
y_test_power_scaled = target_power_scaler.transform(y_test)

target_multi_robust_scalers = [RobustScaler() for _ in range(y.shape[1])]
y_train_multi_robust_scaled = np.zeros_like(y_train)
y_test_multi_robust_scaled = np.zeros_like(y_test)
for i in range(y.shape[1]):
    y_train_multi_robust_scaled[:, i] = target_multi_robust_scalers[i].fit_transform(y_train[:, i].reshape(-1, 1)).ravel()
    y_test_multi_robust_scaled[:, i] = target_multi_robust_scalers[i].transform(y_test[:, i].reshape(-1, 1)).ravel()

target_multi_power_scaler = {i: PowerTransformer(method='yeo-johnson') for i in range(y.shape[1])}
y_train_multi_power_scaled = np.zeros_like(y_train)
y_test_multi_power_scaled = np.zeros_like(y_test)
for i in range(y.shape[1]):
    y_train_multi_power_scaled[:, i] = target_multi_power_scaler[i].fit_transform(y_train[:, i].reshape(-1, 1)).ravel()
    y_test_multi_power_scaled[:, i] = target_multi_power_scaler[i].transform(y_test[:, i].reshape(-1, 1)).ravel()

In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def model_evaluation(y_true, y_pred, model_name=None, debug=False):
    """
    Evaluate the model using MAE, MSE, and R² metrics.
    
    Parameters:
    - y_true: Actual target values
    - y_pred: Predicted target values
    - model_name: Name of the model (optional)
    Returns:
    - mae: Mean Absolute Error
    - mse: Mean Squared Error
    - r2: R² Score
    """

    # Calculate evaluation metrics for each target variable
    mae_scores = []
    mse_scores = []
    r2_scores = []

    for i in range(y_true.shape[1]):
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        mse = mean_squared_error(y_true[:, i], y_pred[:, i])
        r2 = r2_score(y_true[:, i], y_pred[:, i])

        mae_scores.append(mae)
        mse_scores.append(mse)
        r2_scores.append(r2)

        if debug:
            print(f"Target {i+1}:")
            print(f"  MAE: {mae}")
            print(f"  MSE: {mse}")
            print(f"  R²: {r2}")

    # Calculate average scores across all target variables
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} Performance:")
    print(f"MAE: {mae:.4f}, MSE: {mse:.4f}, R²: {r2:.4f}")
    print("-" * 50)

    # Return a dictionary with overall and individual scores
    return {
        "Model": model_name,
        "MAE": mae,
        "MSE": mse,
        "R2": r2,
        "MAE_Scores": mae_scores,
        "MSE_Scores": mse_scores,
        "R2_Scores": r2_scores
    }

def print_predictions(y_pred, y, num=0):
    np.set_printoptions(linewidth=np.inf)  # Set the print options to avoid line breaks
    for i in range(num):
        print(f"Predicted: {y_pred[i]}")
        print(f"Actual:    {y[i]}")
        print("-" * 50)

In [4]:
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
# from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np

# Function to evaluate model performance

# Initialize a list to store model performance
model_performance = []

# Linear Regression
linear_model = MultiOutputRegressor(LinearRegression())
linear_model.fit(X_train_robust_scaled, y_train_multi_power_scaled)
y_pred_linear = linear_model.predict(X_test_robust_scaled)
y_pred_linear_inverse = np.zeros_like(y_pred_linear)
for i in range(y_test.shape[1]):
    y_pred_linear_inverse[:, i] = target_multi_power_scaler[i].inverse_transform(y_pred_linear[:, i].reshape(-1, 1)).ravel()
model_performance.append(model_evaluation(y_test, y_pred_linear_inverse, "Linear Regression"))

# Decision Tree Regressor
decision_tree_model = MultiOutputRegressor(DecisionTreeRegressor(random_state=42))
decision_tree_model.fit(X_train_robust_scaled, y_train_multi_power_scaled)
y_pred_tree = decision_tree_model.predict(X_test_robust_scaled)
y_pred_tree_inverse = np.zeros_like(y_pred_tree)
for i in range(y_test.shape[1]):
    y_pred_tree_inverse[:, i] = target_multi_power_scaler[i].inverse_transform(y_pred_tree[:, i].reshape(-1, 1)).ravel()
model_performance.append(model_evaluation(y_test, y_pred_tree_inverse, "Decision Tree Regressor"))

# Random Forest Regressor
random_forest_model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
random_forest_model.fit(X_train_robust_scaled, y_train_multi_power_scaled)
y_pred_rf = random_forest_model.predict(X_test_robust_scaled)
y_pred_rf_inverse = np.zeros_like(y_pred_rf)
for i in range(y_test.shape[1]):
    y_pred_rf_inverse[:, i] = target_multi_power_scaler[i].inverse_transform(y_pred_rf[:, i].reshape(-1, 1)).ravel()
model_performance.append(model_evaluation(y_test, y_pred_rf_inverse, "Random Forest Regressor"))

# Gradient Boosting Regressor
gradient_boosting_model = MultiOutputRegressor(GradientBoostingRegressor(n_estimators=500, learning_rate=0.1, max_depth=4, random_state=42))
gradient_boosting_model.fit(X_train_robust_scaled, y_train_multi_power_scaled)
y_pred_gb = gradient_boosting_model.predict(X_test_robust_scaled)
y_pred_gb_inverse = np.zeros_like(y_pred_gb)
for i in range(y_test.shape[1]):
    y_pred_gb_inverse[:, i] = target_multi_power_scaler[i].inverse_transform(y_pred_gb[:, i].reshape(-1, 1)).ravel()
model_performance.append(model_evaluation(y_test, y_pred_gb_inverse, "Gradient Boosting Regressor"))

# Support Vector Regression
svr_model = MultiOutputRegressor(SVR(kernel='rbf', C=1.0, epsilon=0.1))
svr_model.fit(X_train_robust_scaled, y_train_multi_power_scaled)
y_pred_svr = svr_model.predict(X_test_robust_scaled)
y_pred_svr_inverse = np.zeros_like(y_pred_svr)
for i in range(y_test.shape[1]):
    y_pred_svr_inverse[:, i] = target_multi_power_scaler[i].inverse_transform(y_pred_svr[:, i].reshape(-1, 1)).ravel()
model_performance.append(model_evaluation(y_test, y_pred_svr_inverse, "Support Vector Regression"))

# K-Nearest Neighbors Regressor
knn_model = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=5))
knn_model.fit(X_train_robust_scaled, y_train_multi_power_scaled)
y_pred_knn = knn_model.predict(X_test_robust_scaled)
y_pred_knn_inverse = np.zeros_like(y_pred_knn)
for i in range(y_test.shape[1]):
    y_pred_knn_inverse[:, i] = target_multi_power_scaler[i].inverse_transform(y_pred_knn[:, i].reshape(-1, 1)).ravel()
model_performance.append(model_evaluation(y_test, y_pred_knn_inverse, "K-Nearest Neighbors Regressor"))

# XGBoost Regressor
# xgb_model = MultiOutputRegressor(XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=4, random_state=42))
# xgb_model.fit(X_train_robust_scaled, y_train_multi_power_scaled)
# y_pred_xgb = xgb_model.predict(X_test_robust_scaled)
# y_pred_xgb_inverse = np.zeros_like(y_pred_xgb)
# for i in range(y_test.shape[1]):
#     y_pred_xgb_inverse[:, i] = target_multi_power_scaler[i].inverse_transform(y_pred_xgb[:, i].reshape(-1, 1)).ravel()
# model_performance.append(model_evaluation(y_test, y_pred_xgb_inverse, "XGBoost Regressor"))

# LightGBM Regressor
lgbm_model = MultiOutputRegressor(LGBMRegressor(n_estimators=500, learning_rate=0.1, max_depth=4, random_state=42))
lgbm_model.fit(X_train_robust_scaled, y_train_multi_power_scaled)
y_pred_lgbm = lgbm_model.predict(X_test_robust_scaled)
y_pred_lgbm_inverse = np.zeros_like(y_pred_lgbm)
for i in range(y_test.shape[1]):
    y_pred_lgbm_inverse[:, i] = target_multi_power_scaler[i].inverse_transform(y_pred_lgbm[:, i].reshape(-1, 1)).ravel()
model_performance.append(model_evaluation(y_test, y_pred_lgbm_inverse, "LightGBM Regressor"))

# Neural Network (MLP Regressor)
mlp_model = MultiOutputRegressor(MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=1000, random_state=42))
mlp_model.fit(X_train_robust_scaled, y_train_multi_power_scaled)
y_pred_mlp = mlp_model.predict(X_test_robust_scaled)
y_pred_mlp_inverse = np.zeros_like(y_pred_mlp)
for i in range(y_test.shape[1]):
    y_pred_mlp_inverse[:, i] = target_multi_power_scaler[i].inverse_transform(y_pred_mlp[:, i].reshape(-1, 1)).ravel()
model_performance.append(model_evaluation(y_test, y_pred_mlp_inverse, "Neural Network (MLP Regressor)"))

# ElasticNet Regression
elasticnet_model = MultiOutputRegressor(ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42))
elasticnet_model.fit(X_train_robust_scaled, y_train_multi_power_scaled)
y_pred_en = elasticnet_model.predict(X_test_robust_scaled)
y_pred_en_inverse = np.zeros_like(y_pred_en)
for i in range(y_test.shape[1]):
    y_pred_en_inverse[:, i] = target_multi_power_scaler[i].inverse_transform(y_pred_en[:, i].reshape(-1, 1)).ravel()
model_performance.append(model_evaluation(y_test, y_pred_en_inverse, "ElasticNet Regression"))

# CatBoost Regressor
catboost_model = MultiOutputRegressor(CatBoostRegressor(iterations=500, learning_rate=0.1, depth=4, random_state=42, verbose=0))
catboost_model.fit(X_train_robust_scaled, y_train_multi_power_scaled)
y_pred_cb = catboost_model.predict(X_test_robust_scaled)
y_pred_cb_inverse = np.zeros_like(y_pred_cb)
for i in range(y_test.shape[1]):
    y_pred_cb_inverse[:, i] = target_multi_power_scaler[i].inverse_transform(y_pred_cb[:, i].reshape(-1, 1)).ravel()
model_performance.append(model_evaluation(y_test, y_pred_cb_inverse, "CatBoost Regressor"))

# Convert the performance list into a DataFrame for better visualization
performance_df = pd.DataFrame(model_performance)
print(performance_df)

# Optional: Save the comparison table to a CSV file
performance_df.to_csv("model_comparison.csv", index=False)


OSError: dlopen(/opt/anaconda3/envs/Thesis/lib/python3.12/site-packages/lightgbm/lib/lib_lightgbm.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib
  Referenced from: <D44045CD-B874-3A27-9A61-F131D99AACE4> /opt/anaconda3/envs/Thesis/lib/python3.12/site-packages/lightgbm/lib/lib_lightgbm.dylib
  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file), '/opt/anaconda3/envs/Thesis/lib/python3.12/lib-dynload/../../libomp.dylib' (no such fil

In [ ]:
# ...existing code...
# prompt: create a funciton for shap analysis, and then iterate all the model, y_pred, y_test from the above models dict

import shap

def shap_analysis(model, X_test, model_name):
    # Check the model type and use the appropriate explainer
    if isinstance(model, (DecisionTreeRegressor, RandomForestRegressor, GradientBoostingRegressor, XGBRegressor, LGBMRegressor)):  
        # Tree-based models
        explainer = shap.TreeExplainer(model)
    elif isinstance(model, (LinearRegression, SVR, KNeighborsRegressor)):  
        # Linear models, SVR, and KNN
        explainer = shap.KernelExplainer(model.predict, X_test)
    elif isinstance(model, ElasticNet):
        # ElasticNet
        explainer = shap.LinearExplainer(model, X_test)
    else:
        raise ValueError(f"Unsupported model type: {type(model)}")

    shap_values = explainer.shap_values(X_test)

    # Plot summary of SHAP values
    shap.summary_plot(shap_values, X_test, feature_names=feature_names, show=False)
    plt.title(f"SHAP Summary Plot - {model_name}")
    plt.show()

# Assuming 'models' is defined as in the provided code.
for model_data in models:
    model = model_data["model"]
    y_pred = model_data["y_pred"]
    y_test = model_data["y_test"]
    model_name = model_data["model_name"]

    # Check if the model has a 'estimators_' attribute (for MultiOutputRegressor)
    if hasattr(model, "estimators_"):
        for i, estimator in enumerate(model.estimators_):
          print(f"SHAP analysis for {model_name}, target {i+1}")
          shap_analysis(estimator, X_test_robust_scaled, f"{model_name} - Target {i+1}")
    else:
        print(f"SHAP analysis for {model_name}")
        shap_analysis(model, X_test_robust_scaled, model_name)


# Sensitivity Analysis for all models
def sensitivity_analysis(model, X_sample, feature_names, variation=0.1):
    sensitivities = {}
    for i, feature in enumerate(feature_names):
        X_varied = X_sample.copy()
        X_varied[:, i] *= (1 + variation)  # Increase feature by 10%
        y_pred_varied = model.predict(X_varied)
        sensitivities[feature] = np.mean(np.abs(y_pred_varied - model.predict(X_sample)))
    return sensitivities

# Bootstrap Uncertainty Estimation for all models
def bootstrap_uncertainty(model, X, n_bootstrap=100):
    predictions = []
    for _ in range(n_bootstrap):
        indices = np.random.choice(range(X.shape[0]), size=X.shape[0], replace=True)
        X_sample = X[indices]
        predictions.append(model.predict(X_sample))
    predictions = np.array(predictions)
    mean_prediction = np.mean(predictions, axis=0)
    std_prediction = np.std(predictions, axis=0)
    return mean_prediction, std_prediction

In [ ]:

# Feature names
feature_names = data.columns[:-7]

# Perform analyses for all models
for model, model_name in [
    (linear_model, "Linear Regression"),
    (decision_tree_model, "Decision Tree Regressor"),
    (random_forest_model, "Random Forest Regressor"),
    (gradient_boosting_model, "Gradient Boosting Regressor"),
    (svr_model, "Support Vector Regression"),
    (knn_model, "K-Nearest Neighbors Regressor"),
    (xgb_model, "XGBoost Regressor"),
    (lgbm_model, "LightGBM Regressor"),
    (mlp_model, "Neural Network (MLP Regressor)"),
    (elasticnet_model, "ElasticNet Regression"),
    (catboost_model, "CatBoost Regressor")
]:
    print(f"Performing SHAP Analysis for {model_name}...")
    shap_analysis(model, X_test_robust_scaled, feature_names, model_name)

    print(f"Performing Sensitivity Analysis for {model_name}...")
    sensitivity = sensitivity_analysis(model, X_test_robust_scaled, feature_names)
    print(f"Sensitivity Analysis ({model_name}):", sensitivity)

    print(f"Performing Bootstrap Uncertainty Estimation for {model_name}...")
    mean_pred, std_pred = bootstrap_uncertainty(model, X_test_robust_scaled)
    print(f"Mean Prediction ({model_name}):", mean_pred)
    print(f"Uncertainty (Standard Deviation):", std_pred)
    print("-" * 50)